In [2]:
pip install pulp

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 15.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [121]:
# setup.py

import pandas as pd
from pulp import LpProblem, LpMinimize

# Initialize the LP problem
lp_problem = LpProblem("StaffSchedulingOptimization", LpMinimize)

# File path for the Excel file
excel_file_path = "/Users/bernardonobrepalma/Desktop/Projeto FIO CUF/DataCUF.xlsx"

# Read data from Excel file into DataFrames
staff_df = pd.read_excel(excel_file_path)
costs_df = pd.read_excel(excel_file_path,2)
overtime_cost_df = pd.read_excel(excel_file_path,3)
min_max_consec_work_hours_df = pd.read_excel(excel_file_path,7)
min_max_week_work_hours_df = pd.read_excel(excel_file_path, 9)
max_overtime_df = pd.read_excel(excel_file_path, 6)
min_staff_allocation_df = pd.read_excel(excel_file_path, 4)
minRestHoursbetweenShifts_df =  pd.read_excel(excel_file_path, 8)

# Add additional sheets as needed

# Extract data from DataFrames
RoleID = staff_df.iloc[3:38, 1].dropna().tolist() 
DeptID = staff_df.iloc[3:11, 5].dropna().tolist()  
Time = staff_df.iloc[4:28, 7].dropna().tolist()
Days = staff_df.iloc[4:11, 10].dropna().tolist()
StaffID = staff_df.iloc[4:743, 16].dropna().tolist()
Shift = staff_df.iloc[3:6, 13].dropna().tolist()
allowedStartHours = staff_df.iloc[14:20, 4].dropna().tolist()
allowedEndHours = staff_df.iloc[22:28, 4].dropna().tolist()

costPerHour = costs_df.iloc[3:41, 1].dropna().tolist()
overtimeCost = overtime_cost_df.iloc[3:41, 1].dropna().tolist()
StaffAvailableperRole = staff_df.iloc[3:41, 2].dropna().tolist()

maxRegularHours = staff_df.iloc[29, 4]  # Assuming 'E31' is in the 31st row, 5th column
maxConsecutiveWorkHours = min_max_consec_work_hours_df.iloc[4:41, 6].dropna().tolist()
minConsecutiveWorkHours = min_max_consec_work_hours_df.iloc[4:41, 1].dropna().tolist()
minRestHoursbetweenShifts = minRestHoursbetweenShifts_df.iloc[4:41:, 1]  # Assuming 'E35' is in the 35th row, 5th column
minHoursPerWeek = min_max_week_work_hours_df.iloc[4:41, 1].dropna().tolist()
maxHoursPerWeek = min_max_week_work_hours_df.iloc[4:41, 6].dropna().tolist()
maxOvertimePerWeek = max_overtime_df.iloc[4:41, 1].dropna().tolist()
minStaffAllocation = min_staff_allocation_df.iloc[4:22, 1:13].values

# Processing


In [122]:

# Load the entire sheet
full_sheet_df = pd.read_excel(excel_file_path, sheet_name="MinStaffAllocation")

# Initialize an empty dictionary to hold the data
min_staff_allocation_dict = {}

# Assuming each role's table is 9 rows high including the header row
rows_per_table = 9

# The number of tables vertically in the sheet
# This depends on the actual layout of your Excel sheet.
# If there are 36 roles and they are laid out in 4 columns, then there would be 9 roles in a column.
num_roles_per_column = 9

# The number of columns each role's table spans, including the role name and time slots
# Adjust this number based on the actual layout.
columns_per_table = full_sheet_df.shape[1]

# Iterate over the roles in the columns
for role_column_start in range(0, columns_per_table, num_roles_per_column):
    # Iterate over the DataFrame in steps of rows_per_table for each role
    for start_row in range(0, full_sheet_df.shape[0], rows_per_table):
        # Slice the DataFrame to get the current role table
        role_table_df = full_sheet_df.iloc[start_row:start_row + rows_per_table, role_column_start:role_column_start + num_roles_per_column]

        # The role name is in the first cell of the table
        role_name = role_table_df.iloc[0, 0]
        if pd.isna(role_name):
            # If the role name is NaN, it means we've reached the end of the role tables
            break

        # Extract the department/time allocation matrix, skipping the first header row
        # and the first column which contains the department names
        allocation_matrix = role_table_df.iloc[1:, 1:]

        # Fill NaN values with 0
        allocation_matrix = allocation_matrix.fillna(0)

        # Convert the matrix to a numpy array or to a list of lists
        allocation_matrix = allocation_matrix.values

        # Store the matrix in the dictionary under the role name
        min_staff_allocation_dict[role_name] = allocation_matrix


In [123]:
min_staff_allocation_df

,Minimum Staff Allocation (Departments x Roles),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hora,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Department / Role,Assistentes de saúde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Atendimento Permanente,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,Consultas,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,Cuidados Paliativos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,Internamentos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,Atendimento e Consultas de Enfermagem,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:

# Load the entire sheet
full_sheet_df = pd.read_excel(excel_file_path, sheet_name="MinStaffAllocation")

# Initialize an empty dictionary to hold the data
min_staff_allocation_dict = {}

# The number of rows including the header row, adjust if different
rows_per_role_table = 9

# The number of columns each role's table spans, including the role name and time slots
columns_per_role_table = full_sheet_df.shape[1]  # Assuming the table spans all columns

# Iterate over the DataFrame in steps of rows_per_role_table to read each role's table
for start_row in range(6, full_sheet_df.shape[0], rows_per_role_table):  # Starting at row index 6 assuming headers are in row 5 (index 4)
    # Slice the DataFrame to get the current role table
    role_table_df = full_sheet_df.iloc[start_row:start_row + rows_per_role_table, :columns_per_role_table]

    # The role name is in the first cell of the table, in column A
    role_name = role_table_df.iloc[0, 0]
    if pd.isna(role_name):
        # If the role name is NaN, it means we've reached the end of the role tables
        break

    # Extract the department/time allocation matrix, skipping the first row and first column
    allocation_matrix = role_table_df.iloc[1:, 1:]

    # Fill NaN values with 0
    allocation_matrix = allocation_matrix.fillna(0)

    # Convert the matrix to a numpy array or a list of lists
    allocation_matrix = allocation_matrix.to_numpy()

    # Store the matrix in the dictionary under the role name
    min_staff_allocation_dict[role_name] = allocation_matrix

/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_21360/2892888244.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allocation_matrix = allocation_matrix.fillna(0)


In [133]:
full_sheet_df = pd.read_excel(excel_file_path, sheet_name="MinStaffAllocation")

# Initialize an empty dictionary to hold the data
min_staff_allocation_dict = {}

# List of starting rows for each table
start_rows = [i for i in range(6, 439, 12)]  # Fill in all the starting rows



# Number of rows for each table, assuming it's consistent for all roles
rows_per_table = 9  # Adjust if different for each role

# Iterate over each starting row and role ID
for start_row, role_id in zip(start_rows, RoleID):
    # Extract the department/time allocation matrix for the current role
    # Skip the first row which is the header and the first column which is the time header
    allocation_matrix = full_sheet_df.iloc[start_row -2:start_row + rows_per_table, 1:]
    print(allocation_matrix)
    # Fill NaN values with 0
    allocation_matrix = allocation_matrix.fillna(0)

    # Convert the matrix to a numpy array
    allocation_matrix = allocation_matrix.to_numpy()

    # Store the matrix in the dictionary under the role ID
    min_staff_allocation_dict[role_id] = allocation_matrix


   Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
4         NaN        NaN        NaN        NaN        NaN        NaN   
5         NaN        NaN        NaN        NaN        NaN        NaN   
6         NaN        NaN        NaN        NaN        NaN        NaN   
7           0          0          0          0          0          0   
8         NaN        NaN        NaN        NaN        NaN        NaN   
9         NaN        NaN        NaN        NaN        NaN        NaN   
10        NaN        NaN        NaN        NaN        NaN        NaN   
11          0          0          0          0          0          0   
12        NaN        NaN        NaN        NaN        NaN        NaN   
13        NaN        NaN        NaN        NaN        NaN        NaN   
14   01:00:00   02:00:00   03:00:00   04:00:00   05:00:00   06:00:00   

   Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10  ... Unnamed: 71 Unnamed: 72  \
4         NaN        NaN        NaN         NaN  ...   

/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_21360/575415814.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allocation_matrix = allocation_matrix.fillna(0)
/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_21360/575415814.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allocation_matrix = allocation_matrix.fillna(0)
/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_21360/575415814.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. C

In [3]:
# setup.py
import pandas as pd
from pulp import LpProblem, LpMinimize

lp_problem = LpProblem("StaffSchedulingOptimization", LpMinimize)

# File path for the Excel file
excel_file_path = "/Users/bernardonobrepalma/Desktop/Projeto FIO CUF/DataCUF.xlsx"

# Read data from Excel file into DataFrames
staff_df = pd.read_excel(excel_file_path)
costs_df = pd.read_excel(excel_file_path,2)
overtime_cost_df = pd.read_excel(excel_file_path,3)
min_max_consec_work_hours_df = pd.read_excel(excel_file_path,7)
min_max_week_work_hours_df = pd.read_excel(excel_file_path, 9)
max_overtime_df = pd.read_excel(excel_file_path, 6)
min_staff_allocation_df = pd.read_excel(excel_file_path, 4)
max_staff_allocation_df = pd.read_excel(excel_file_path, 5)
minRestHoursbetweenShifts_df =  pd.read_excel(excel_file_path, 8)


# Extract data from DataFrames
RoleID = staff_df.iloc[3:38, 1].dropna().tolist() 
DeptID = staff_df.iloc[3:11, 5].dropna().tolist()  
Time = staff_df.iloc[4:28, 7].dropna().tolist()
Days = staff_df.iloc[4:11, 10].dropna().tolist()
StaffID = staff_df.iloc[4:743, 16].dropna().tolist()
Shift = staff_df.iloc[3:6, 13].dropna().tolist()
allowedStartHours = staff_df.iloc[14:20, 4].dropna().tolist()
allowedEndHours = staff_df.iloc[22:28, 4].dropna().tolist()

costPerHour = costs_df.iloc[3:41, 1].dropna().tolist()
overtimeCost = overtime_cost_df.iloc[3:41, 1].dropna().tolist()
StaffAvailableperRole = staff_df.iloc[3:41, 2].dropna().tolist()

maxRegularHours = staff_df.iloc[29, 4] 
maxConsecutiveWorkHours = min_max_consec_work_hours_df.iloc[4:41, 6].dropna().tolist()
minConsecutiveWorkHours = min_max_consec_work_hours_df.iloc[4:41, 1].dropna().tolist()
minRestHoursbetweenShifts = minRestHoursbetweenShifts_df.iloc[4:41:, 1]  
minHoursPerWeek = min_max_week_work_hours_df.iloc[4:41, 1].dropna().tolist()
maxHoursPerWeek = min_max_week_work_hours_df.iloc[4:41, 6].dropna().tolist()
maxOvertimePerWeek = max_overtime_df.iloc[4:41, 1].dropna().tolist()



min_staff_allocation_dict = {}
max_staff_allocation_dict = {}

# Start rows for each role's table (36 roles, each table with 10 rows, starting from row 4)
start_rows = [4 + i * 10 for i in range(36)] 
rows_per_table = 8  # Only 8 rows with information per role

for start_row, role_id in zip(start_rows, RoleID):
    # Selecting the range for each role's table
    allocation_matrix = max_staff_allocation_df.iloc[start_row:start_row + rows_per_table, 1:]
    allocation_matrix2 = max_staff_allocation_df.iloc[start_row:start_row + rows_per_table, 1:]
    
    # Filling NaN values with 0 and converting DataFrame to numpy array
    allocation_matrix = allocation_matrix.fillna(0).to_numpy()
    allocation_matrix2 = allocation_matrix2.fillna(0).to_numpy()
    min_staff_allocation_dict[role_id] = allocation_matrix
    max_staff_allocation_dict[role_id] = allocation_matrix2


/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_98011/3307117154.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allocation_matrix = allocation_matrix.fillna(0).to_numpy()
/var/folders/jg/b65zhgc145d11fyxs84_2ybw0000gn/T/ipykernel_98011/3307117154.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allocation_matrix2 = allocation_matrix2.fillna(0).to_numpy()


{1: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0